<a href="https://colab.research.google.com/github/adsamardeep/Twitter_Sentiment_Analysis/blob/master/SentimentsMotherDay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk
import pandas as pd
#from emoticons import EmoticonDetector
import re as regex
import numpy as np
#import plotly
#from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
#plotly.offline.init_notebook_mode()

import seaborn as sns
#import plotly
#import cufflinks as cf
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


True

# Import the Data

In [0]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

In [0]:
train_data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [0]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3235 non-null   float64
 1   original_text    3235 non-null   object 
 2   lang             3231 non-null   object 
 3   retweet_count    3231 non-null   object 
 4   original_author  3235 non-null   object 
 5   sentiment_class  3235 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 151.8+ KB


In [0]:
test_data.head()

,id,original_text,lang,retweet_count,original_author
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg


In [0]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387 entries, 0 to 1386
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1387 non-null   float64
 1   original_text    1387 non-null   object 
 2   lang             1387 non-null   object 
 3   retweet_count    1386 non-null   object 
 4   original_author  1387 non-null   object 
dtypes: float64(1), object(4)
memory usage: 54.3+ KB


In [0]:
train_data.isnull().sum(axis = 0)

id                 0
original_text      0
lang               4
retweet_count      4
original_author    0
sentiment_class    0
dtype: int64

In [0]:
train_data = train_data.fillna(0)
train_data.isnull().sum(axis = 0)

id                 0
original_text      0
lang               0
retweet_count      0
original_author    0
sentiment_class    0
dtype: int64

In [0]:
retweet_replace = ['en',' BLOOD OF PROTECTION AROUND YOU GUYS',' Shop &lt',' have no other means of occasional transportation! Mary.'," He's told me I don't understand what it means to be a special agent LOL"]
for i in retweet_replace:
  train_data['retweet_count'] = train_data['retweet_count'].apply(lambda x: 0 if x == i else x)

In [0]:
train_data['retweet_count'].unique()

array(['0', '1', '3', 0, '2', '12', '4', '0.2255471156', '5', '24',
       '0.704111013', '7', '20', '-0.7272841814', '-0.5391119831', '11',
       '16', '0.2705365837', '32', '0.6908800807', '6', '0.2356132752',
       '-0.5267869213', '-0.4666353157', '-0.8614161431', '9', '8', '17',
       '-0.9319417873', '-0.4199361695', '37', '10', '0.1078812656', '27',
       '0.9846158924', '33', '13', '15', '118', '59', '-0.8020153269',
       '0.8574897643', '14', '-0.3643299313', '0.1005978336', '21', '19',
       '0.4005356538', '-0.7229893804', '18', '-0.9278231369',
       '-0.1417566007', '0.1869516081', '23', '-0.9386981134', '25',
       '-0.3101107397', '0.9930106132', '35', '0.6610057615', '61',
       '-0.057256529', '28', '96', '0.3091754197', '45', '0.0375273346',
       '0.7006763228'], dtype=object)

# Data Cleaning

In [0]:
def clean_tweets(tweet):
    
    # remove URL
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Remove usernames
    tweet = re.sub(r"@[^\s]+[\s]?",'',tweet)
    
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)
    
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)
    
    return tweet

In [0]:
train_data['original_text'] = train_data['original_text'].apply(clean_tweets)

In [0]:
train_data['original_text'].head()

0    Happy MothersDay to all you amazing mothers ou...
1    Happy Mothers Day Mum  Im sorry I cant be ther...
2    Happy mothers day To all This doing a mothers ...
3    Happy mothers day to this beautiful womanroyal...
4    Remembering the  most amazing ladies who made ...
Name: original_text, dtype: object

# Tokenization & stemming

In [0]:
# Function which directly tokenize the tweet data
from nltk.tokenize import TweetTokenizer

tt = TweetTokenizer()
train_data['original_text'].apply(tt.tokenize)

0       [Happy, MothersDay, to, all, you, amazing, mot...
1       [Happy, Mothers, Day, Mum, Im, sorry, I, cant,...
2       [Happy, mothers, day, To, all, This, doing, a,...
3       [Happy, mothers, day, to, this, beautiful, wom...
4       [Remembering, the, most, amazing, ladies, who,...
                              ...                        
3230    [To, all, my, sisters, my, sisters, in, law, a...
3231    [Happy, Mothers, Day, to, all, the, Mums, Step...
3232    [Happy, Mothers, Day, to, the, craziest, woman...
3233    [Happy, Mothers, Day, to, my, amazing, wife, W...
3234    [Wishing, you, all, a, safe, happy, Mothers, D...
Name: original_text, Length: 3235, dtype: object

In [0]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [0]:
def tokenize(text):
    return word_tokenize(text)

def stemming(words):
    stem_words = []
    for w in words:
        w = ps.stem(w)
        stem_words.append(w)
    
    return stem_words

In [0]:
# apply tokenize function
train_data['text'] = train_data['original_text'].apply(tokenize)

In [0]:
# apply steming function
train_data['tokenized'] = train_data['text'].apply(stemming)

In [0]:
train_data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class,text,tokenized
0,1.245025e+18,Happy MothersDay to all you amazing mothers ou...,en,0,BeenXXPired,0,"[Happy, MothersDay, to, all, you, amazing, mot...","[happi, mothersday, to, all, you, amaz, mother..."
1,1.245759e+18,Happy Mothers Day Mum Im sorry I cant be ther...,en,1,FestiveFeeling,0,"[Happy, Mothers, Day, Mum, Im, sorry, I, cant,...","[happi, mother, day, mum, Im, sorri, I, cant, ..."
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1,"[Happy, mothers, day, To, all, This, doing, a,...","[happi, mother, day, To, all, thi, do, a, moth..."
3,1.244803e+18,Happy mothers day to this beautiful womanroyal...,en,0,Queenuchee,0,"[Happy, mothers, day, to, this, beautiful, wom...","[happi, mother, day, to, thi, beauti, womanroy..."
4,1.244876e+18,Remembering the most amazing ladies who made ...,en,0,brittan17446794,-1,"[Remembering, the, most, amazing, ladies, who,...","[rememb, the, most, amaz, ladi, who, made, me,..."


# Wordlist

In [0]:
words = Counter()
for idx in train_data.index:
    words.update(train_data.loc[idx, "text"])

words.most_common(5)

[('to', 3673),
 ('the', 2850),
 ('Happy', 2738),
 ('and', 2445),
 ('Mothers', 2379)]

In [0]:
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
whitelist = ["n't", "not"]
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]
words.most_common(5)

[('Happy', 2738),
 ('Mothers', 2379),
 ('Day', 1964),
 ('day', 1606),
 ('mothers', 1431)]

In [0]:
def word_list(processed_data):
    #print(processed_data)
    min_occurrences=3 
    max_occurences=500 
    stopwords=nltk.corpus.stopwords.words("english")
    whitelist = ["n't","not"]
    wordlist = []
    
    whitelist = whitelist if whitelist is None else whitelist
    #print(whitelist)
    '''
    import os
    if os.path.isfile("wordlist.csv"):
        word_df = pd.read_csv("wordlist.csv")
        word_df = word_df[word_df["occurrences"] > min_occurrences]
        wordlist = list(word_df.loc[:, "word"])
        #return
    '''
    words = Counter()
    for idx in processed_data.index:
        words.update(processed_data.loc[idx, "text"])

    for idx, stop_word in enumerate(stopwords):
        if stop_word not in whitelist:
            del words[stop_word]
    #print(words)

    word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                 "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                           columns=["word", "occurrences"])
    #print(word_df)
    word_df.to_csv("wordlist.csv", index_label="idx")
    wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
    #print(wordlist)

In [0]:
word_list(train_data)

In [0]:
words = pd.read_csv("wordlist.csv")

# Bag of Words

In [0]:
import os

In [0]:
wordlist= []
if os.path.isfile("wordlist.csv"):
    word_df = pd.read_csv("wordlist.csv")
    word_df = word_df[word_df["occurrences"] > 3]
    wordlist = list(word_df.loc[:, "word"])

label_column = ["label"]
columns = label_column + list(map(lambda word: str(word) + '_bow',wordlist))
labels = []
rows = []
for idx in train_data.index:
    current_row = []
    
    # add label
    current_label = train_data.loc[idx, "sentiment_class"]
    labels.append(current_label)
    current_row.append(current_label)

    # add bag-of-words
    tokens = set(train_data.loc[idx, "text"])
    for _, word in enumerate(wordlist):
        current_row.append(1 if word in tokens else 0)

    rows.append(current_row)

data_model = pd.DataFrame(rows, columns=columns)
data_labels = pd.Series(labels)


bow = data_model

In [0]:
bow.head()

,label,today_bow,us_bow,We_bow,mother_bow,amazing_bow,not_bow,wonderful_bow,Mum_bow,To_bow,world_bow,one_bow,Mums_bow,know_bow,hope_bow,best_bow,like_bow,safe_bow,beautiful_bow,much_bow,You_bow,wish_bow,lovely_bow,family_bow,time_bow,Love_bow,everyone_bow,DAY_bow,Thank_bow,MOTHERS_bow,Im_bow,always_bow,A_bow,see_bow,every_bow,HAPPY_bow,The_bow,special_bow,happymothersday_bow,women_bow,...,image_bow,Distancing_bow,dream_bow,wrong_bow,Maria_bow,harder_bow,slightly_bow,feels_bow,keepsafe_bow,motherlylove_bow,AMAZING_bow,inthismomentofficial_bow,album_bow,orders_bow,street_bow,cheer_bow,Really_bow,arrived_bow,souls_bow,GOD_bow,RIP_bow,Dinner_bow,healthcare_bow,emotional_bow,pet_bow,potential_bow,Little_bow,vision_bow,FOR_bow,heartbeat_bow,black_bow,Maybe_bow,gem_bow,werent_bow,Markel_bow,countries_bow,calling_bow,Prince_bow,practicing_bow,calls_bow
0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Classification

In [0]:
import random
seed = 777
random.seed(seed)

In [0]:
def test_classifier(X_train, y_train, X_test, y_test, classifier):
    log("")
    log("---------------------------------------------------------")
    log("Testing " + str(type(classifier).__name__))
    now = time()
    list_of_labels = sorted(list(set(y_train)))
    model = classifier.fit(X_train, y_train)
    log("Learing time {0}s".format(time() - now))
    now = time()
    predictions = model.predict(X_test)
    log("Predicting time {0}s".format(time() - now))

    # Calculate Accuracy, Precision, recall
    
    precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    
    log("=================== Results ===================")
    log("            Negative     Neutral     Positive")
    log("F1       " + str(f1))
    log("Precision" + str(precision))
    log("Recall   " + str(recall))
    log("Accuracy " + str(accuracy))
    log("===============================================")

    return precision, recall, accuracy, f1

def log(x):
    #can be used to write to log file
    print(x)

# Experiment 1: BOW + Naive Bayes

In [0]:
from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow['label'], test_size=0.3, random_state=seed)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, BernoulliNB())


---------------------------------------------------------
Testing BernoulliNB
Learing time 0.10297536849975586s
Predicting time 0.04215860366821289s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.1627907  0.6193969  0.20485175]
Precision[0.24778761 0.52054795 0.296875  ]
Recall   [0.12121212 0.76458753 0.1563786 ]
Accuracy 0.4593202883625129


In [0]:
def cv(classifier, X_train, y_train):
    log("===============================================")
    classifier_name = str(type(classifier).__name__)
    now = time()
    log("Crossvalidating " + classifier_name + "...")
    accuracy = [cross_val_score(classifier, X_train, y_train, cv=8, n_jobs=-1)]
    log("Crosvalidation completed in {0}s".format(time() - now))
    log("Accuracy: " + str(accuracy[0]))
    log("Average accuracy: " + str(np.array(accuracy[0]).mean()))
    log("===============================================")
    return accuracy

# Add Extra Features

In [0]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

In [0]:
train_data = train_data.fillna(0)
train_data.isnull().sum(axis = 0)

id                 0
original_text      0
lang               0
retweet_count      0
original_author    0
sentiment_class    0
dtype: int64

In [0]:
train_data['retweet_count'].unique()

array(['0', '1', '3', 'en', '2', '12', '4', 0, '0.2255471156', '5', '24',
       '0.704111013', '7', '20', '-0.7272841814', '-0.5391119831', '11',
       '16', ' BLOOD OF PROTECTION AROUND YOU GUYS', '0.2705365837', '32',
       '0.6908800807', '6', '0.2356132752', '-0.5267869213',
       '-0.4666353157', '-0.8614161431', '9', '8', '17', '-0.9319417873',
       '-0.4199361695', '37', '10', '0.1078812656', '27', '0.9846158924',
       '33', ' Shop &lt', '13', '15', '118', '59', '-0.8020153269',
       '0.8574897643', '14', '-0.3643299313', '0.1005978336', '21', '19',
       '0.4005356538', '-0.7229893804', '18', '-0.9278231369',
       '-0.1417566007', '0.1869516081', '23', '-0.9386981134', '25',
       '-0.3101107397', '0.9930106132', '35',
       ' have no other means of occasional transportation! Mary.',
       '0.6610057615', '61',
       " He's told me I don't understand what it means to be a special agent LOL",
       '-0.057256529', '28', '96', '0.3091754197', '45', '0.0375273346

In [0]:
retweet_replace = ['en',' BLOOD OF PROTECTION AROUND YOU GUYS',' Shop &lt',' have no other means of occasional transportation! Mary.'," He's told me I don't understand what it means to be a special agent LOL"]
for i in retweet_replace:
  train_data['retweet_count'] = train_data['retweet_count'].apply(lambda x: 0 if x == i else x)

In [0]:
train_data['retweet_count'].unique()

array(['0', '1', '3', 0, '2', '12', '4', '0.2255471156', '5', '24',
       '0.704111013', '7', '20', '-0.7272841814', '-0.5391119831', '11',
       '16', '0.2705365837', '32', '0.6908800807', '6', '0.2356132752',
       '-0.5267869213', '-0.4666353157', '-0.8614161431', '9', '8', '17',
       '-0.9319417873', '-0.4199361695', '37', '10', '0.1078812656', '27',
       '0.9846158924', '33', '13', '15', '118', '59', '-0.8020153269',
       '0.8574897643', '14', '-0.3643299313', '0.1005978336', '21', '19',
       '0.4005356538', '-0.7229893804', '18', '-0.9278231369',
       '-0.1417566007', '0.1869516081', '23', '-0.9386981134', '25',
       '-0.3101107397', '0.9930106132', '35', '0.6610057615', '61',
       '-0.057256529', '28', '96', '0.3091754197', '45', '0.0375273346',
       '0.7006763228'], dtype=object)

In [0]:
def add_extra_feature(df, tweet_column):
    
    # Print Number of Exclamation
    #length_of_excl = (len(re.findall(r'!', string)))
    df['number_of_exclamation'] = tweet_column.apply(lambda x: (len(re.findall(r'!', x))))
    
    # Number of ?
    #length_of_questionmark = (len(re.findall(r'?', string)))
    df['number_of_questionmark'] = tweet_column.apply(lambda x: (len(re.findall(r'[?]', x))))
    
    # Number of #
    df['number_of_hashtag'] = tweet_column.apply(lambda x: (len(re.findall(r'#', x))))
    
    # Number of @
    df['number_of_mention'] = tweet_column.apply(lambda x: (len(re.findall(r'@', x))))
    
    # Number of Quotes
    df['number_of_quotes'] = tweet_column.apply(lambda x: (len(re.findall(r"'", x))))

    # Number if underscore
    df['number_of_underscore'] = tweet_column.apply(lambda x: (len(re.findall(r'_', x))))
    
    
    #print((txt.split(" "), row))
    #print(row.split())

In [0]:
# pass the train_data into add_extra_feature function
add_extra_feature(train_data, train_data["original_text"])

# ADD EMOTICONS

In [0]:
## Emoticon Detector

class EmoticonDetector:
    emoticons = {}

    def __init__(self, emoticon_file="emoticons.txt"):
        from pathlib import Path
        content = Path(emoticon_file).read_text()
        positive = True
        for line in content.split("\n"):
            if "positive" in line.lower():
                positive = True
                continue
            elif "negative" in line.lower():
                positive = False
                continue

            self.emoticons[line] = positive

    def is_positive(self, emoticon):
        if emoticon in self.emoticons:
            return self.emoticons[emoticon]
        return False

    def is_emoticon(self, to_check):
        return to_check in self.emoticons

In [0]:
ed = EmoticonDetector()

In [0]:
processed_data = train_data.copy()

def add_column(column_name, column_content):
    processed_data.loc[:, column_name] = pd.Series(column_content, index=processed_data.index)

def count_by_lambda(expression, word_array):
    return len(list(filter(expression, word_array)))

add_column("splitted_text", map(lambda txt: txt.split(" "), processed_data["original_text"]))

positive_emo = list(
    map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and ed.is_positive(word), txt),
        processed_data["splitted_text"]))
add_column("number_of_positive_emo", positive_emo)

negative_emo = list(map(
    lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and not ed.is_positive(word), txt),
    processed_data["splitted_text"]))

add_column("number_of_negative_emo", negative_emo)

In [0]:
train_data['number_of_positive_emo'] = positive_emo
train_data['number_of_negative_emo'] = negative_emo

# Prepare training data for model

In [0]:
train_data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class,number_of_exclamation,number_of_questionmark,number_of_hashtag,number_of_mention,number_of_quotes,number_of_underscore,number_of_positive_emo,number_of_negative_emo
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0,1,0,2,0,2,0,0,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0,1,0,0,0,4,0,0,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1,0,0,1,0,0,0,0,0
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0,0,0,2,0,0,0,0,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1,3,0,0,0,0,0,0,0


In [0]:
# apply the clean tweet function
train_data['original_text'] = train_data['original_text'].apply(clean_tweets)

In [0]:
## Tokenize & stemming data
train_data['text'] = train_data['original_text'].apply(tokenize)
train_data['tokenized'] = train_data['text'].apply(stemming)

In [0]:
# delete old file first
# wordlist
word_list(train_data)

In [0]:
wordlist= []
if os.path.isfile("wordlist.csv"):
    word_df = pd.read_csv("wordlist.csv")
    word_df = word_df[word_df["occurrences"] > 3]
    wordlist = list(word_df.loc[:, "word"])

label_column = ["label"]
columns = label_column + list(map(lambda w: str(w) + "_bow",wordlist))
labels = []
rows = []
for idx in train_data.index:
    current_row = []
        # add label
    current_label = train_data.loc[idx, "sentiment_class"]
    labels.append(current_label)
    current_row.append(current_label)

    # add bag-of-words
    tokens = set(train_data.loc[idx, "text"])
    for _, word in enumerate(wordlist):
        current_row.append(1 if word in tokens else 0)

    rows.append(current_row)

data_model = pd.DataFrame(rows, columns=columns)
data_labels = pd.Series(labels)

In [0]:
dat1 = train_data
dat2 = data_model

dat1 = dat1.reset_index(drop=True)
dat2 = dat2.reset_index(drop=True)

data_model = dat1.join(dat2)

In [0]:
train_data.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'sentiment_class', 'number_of_exclamation', 'number_of_questionmark',
       'number_of_hashtag', 'number_of_mention', 'number_of_quotes',
       'number_of_underscore', 'number_of_positive_emo',
       'number_of_negative_emo', 'text', 'tokenized'],
      dtype='object')

In [0]:
## Drop the columns in data_model
data_model = data_model.drop(columns=['id', 'original_text', 'lang', 'original_author',
       'sentiment_class', 'text', 'tokenized'], axis=1)

In [0]:
data_model.columns

Index(['retweet_count', 'number_of_exclamation', 'number_of_questionmark',
       'number_of_hashtag', 'number_of_mention', 'number_of_quotes',
       'number_of_underscore', 'number_of_positive_emo',
       'number_of_negative_emo', 'label',
       ...
       'black_bow', 'Maybe_bow', 'gem_bow', 'werent_bow', 'Markel_bow',
       'countries_bow', 'calling_bow', 'Prince_bow', 'practicing_bow',
       'calls_bow'],
      dtype='object', length=1828)

In [0]:
data_model['retweet_count'] = data_model['retweet_count'].astype(float)

# Experiment 2: Added feature + Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(data_model.drop(columns='label',axis=1),data_model['label'] , test_size=0.3)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, RandomForestClassifier(random_state=seed,n_estimators=403,n_jobs=-1))


---------------------------------------------------------
Testing RandomForestClassifier
Learing time 11.087128400802612s
Predicting time 0.31198644638061523s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.125      0.68069666 0.07971014]
Precision[0.27272727 0.54982415 0.21153846]
Recall   [0.08108108 0.89333333 0.04910714]
Accuracy 0.5128733264675592


In [0]:
rf_acc = cv(RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed),data_model.drop(columns='label',axis=1), data_model['label'])

Crossvalidating RandomForestClassifier...
Crosvalidation completed in 132.67884182929993s
Accuracy: [0.47901235 0.51358025 0.49382716 0.50742574 0.48019802 0.51485149
 0.4950495  0.4950495 ]
Average accuracy: 0.4973742513140203


# Experiment 3: Added Feature + XGBoost

In [0]:
!pip install xgboost
from xgboost import XGBClassifier as XGBoostClassifier

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_model.drop(columns='label',axis=1),data_model['label'] , test_size=0.3)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, XGBoostClassifier(seed=seed))


---------------------------------------------------------
Testing XGBClassifier
Learing time 25.302754402160645s
Predicting time 0.07223892211914062s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.02542373 0.69135802 0.03225806]
Precision[0.2        0.53903743 0.19047619]
Recall   [0.01357466 0.96367113 0.01762115]
Accuracy 0.5262615859938208


In [0]:
xgb_acc = cv(XGBoostClassifier(seed=seed),data_model.drop(columns='label',axis=1), data_model['label'])

Crossvalidating XGBClassifier...
Crosvalidation completed in 186.26300859451294s
Accuracy: [0.52592593 0.52839506 0.51358025 0.51485149 0.51732673 0.51732673
 0.52227723 0.51732673]
Average accuracy: 0.5196262681823738


# Experiment 4: Added Feature + Naive Bayes

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_model.drop(columns='label',axis=1),data_model['label'] , test_size=0.3)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, BernoulliNB())


---------------------------------------------------------
Testing BernoulliNB
Learing time 0.16372966766357422s
Predicting time 0.05246543884277344s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.11363636 0.60576923 0.10526316]
Precision[0.17391304 0.5046729  0.1682243 ]
Recall   [0.08438819 0.75751503 0.07659574]
Accuracy 0.42842430484037075


In [0]:
nb_acc = cv(BernoulliNB(), data_model.drop(columns='label',axis=1), data_model['label'])

Crossvalidating BernoulliNB...
Crosvalidation completed in 1.4047167301177979s
Accuracy: [0.45925926 0.44691358 0.43209877 0.47772277 0.42079208 0.44554455
 0.41584158 0.43564356]
Average accuracy: 0.4417270199242146


# Finding best parameters


In [0]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            log("Model with rank: {0}".format(i))
            log("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            log("Parameters: {0}".format(results['params'][candidate]))
            log("")

def best_fit(X_train, y_train, n_iter=5):
    
    parameters = {
        "n_estimators":[103,201, 403],
        "max_depth":[3,10,15, 30],
        "objective":["multi:softmax","binary:logistic"],
        "learning_rate":[0.05, 0.1, 0.15, 0.3]
    }

    rand_search = RandomizedSearchCV(XGBoostClassifier(seed=seed),param_distributions=parameters,
                                     n_iter=n_iter,scoring="accuracy",
                                     n_jobs=-1,cv=8)

    import time as ttt
    now = time()
    log(ttt.ctime())
    rand_search.fit(X_train, y_train)
    report(rand_search.cv_results_, 10)
    log(ttt.ctime())
    log("Search took: " + str(time() - now))

In [0]:
best_fit(data_model.drop(columns='label',axis=1), data_model['label'], n_iter=10)

Tue Jun  9 19:21:50 2020


# Test Data

In [0]:
test_data.head()

,id,original_text,lang,retweet_count,original_author
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg


In [0]:
test_data.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author'], dtype='object')

In [0]:
# remove the tweets which contains Not available
test_data = test_data[test_data['original_text'] != "Not Available"]

In [0]:
test_data = test_data.fillna(0)
test_data.isnull().sum(axis = 0)

id                 0
original_text      0
lang               0
retweet_count      0
original_author    0
dtype: int64

In [0]:
test_data['retweet_count'].unique()

array(['0', '1', '2', 'en', '-0.9676057189',
       'encouraging- thanks. more2come', '5', '13', '3', '17', '31', '14',
       '6', '0.5309002891', '26', '4', '44', '58', '-0.1277872725',
       '0.4963592171', 0, '16', '-0.8606548851', '15', '11', '29', '7',
       '8', '0.0182429262', '19', '0.6782208946', '0.5248572384',
       '-0.9417957712', '40', '35', ' Shop &lt', '-0.7073840905', '10',
       ' Day//Gift for Moms//Personalized Shirt', '47', '33',
       '0.4802438451'], dtype=object)

In [0]:
retweet_replace = ['en','encouraging- thanks. more2come',' Shop &lt',' Day//Gift for Moms//Personalized Shirt']
for i in retweet_replace:
  test_data['retweet_count'] = test_data['retweet_count'].apply(lambda x: 0 if x == i else x)

In [0]:
test_data['retweet_count'].unique()

array(['0', '1', '2', 0, '-0.9676057189', '5', '13', '3', '17', '31',
       '14', '6', '0.5309002891', '26', '4', '44', '58', '-0.1277872725',
       '0.4963592171', '16', '-0.8606548851', '15', '11', '29', '7', '8',
       '0.0182429262', '19', '0.6782208946', '0.5248572384',
       '-0.9417957712', '40', '35', '-0.7073840905', '10', '47', '33',
       '0.4802438451'], dtype=object)

In [0]:
processed_data = test_data.copy()

def add_column(column_name, column_content):
    processed_data.loc[:, column_name] = pd.Series(column_content, index=processed_data.index)

def count_by_lambda(expression, word_array):
    return len(list(filter(expression, word_array)))

add_column("splitted_text", map(lambda txt: txt.split(" "), processed_data["original_text"]))

positive_emo = list(
    map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and ed.is_positive(word), txt),
        processed_data["splitted_text"]))
add_column("number_of_positive_emo", positive_emo)

negative_emo = list(map(
    lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and not ed.is_positive(word), txt),
    processed_data["splitted_text"]))

add_column("number_of_negative_emo", negative_emo)

In [0]:
# Drop null values
test_data = test_data.dropna() 

# add extra features
add_extra_feature(test_data, test_data['original_text'])

# add emoticon
test_data['number_of_positive_emo'] = positive_emo
test_data['number_of_negative_emo'] = negative_emo

# Clean tweets
test_data['original_text'] = test_data['original_text'].apply(clean_tweets)

## Tokenize data
test_data['text'] = test_data['original_text'].apply(tokenize)
test_data['tokenized'] = test_data['text'].apply(stemming)

In [0]:
# wordlist
#word_list(test_data)

In [0]:
## BAG OF WORDS
wordlist= []
if os.path.isfile("wordlist.csv"):
    word_df = pd.read_csv("wordlist.csv")
    word_df = word_df[word_df["occurrences"] > 3]
    wordlist = list(word_df.loc[:, "word"])

label_column = ["label"]
columns = label_column + list(map(lambda w: str(w) + "_bow",wordlist))
labels = []
rows = []
for idx in test_data.index:
    current_row = []
    current_row.append('retweet_count')
        # add label
    #current_label = test_data.loc[idx, "original_text"]
    #labels.append(current_label)
    #current_row.append(current_label)

    # add bag-of-words
    tokens = set(test_data.loc[idx, "text"])
    for _, word in enumerate(wordlist):
        current_row.append(1 if word in tokens else 0)

    rows.append(current_row)

test_data_model = pd.DataFrame(rows, columns=columns)
data_labels = pd.Series(labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [0]:
dat1 = test_data
dat2 = test_data_model

dat1 = dat1.reset_index(drop=True)
dat2 = dat2.reset_index(drop=True)

test_data_model = dat1.join(dat2)

In [0]:
test_model = pd.DataFrame()
test_model['id'] = test_data_model['id']

In [0]:
test_data_model.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'number_of_exclamation', 'number_of_questionmark', 'number_of_hashtag',
       'number_of_mention', 'number_of_quotes',
       ...
       'black_bow', 'Maybe_bow', 'gem_bow', 'werent_bow', 'Markel_bow',
       'countries_bow', 'calling_bow', 'Prince_bow', 'practicing_bow',
       'calls_bow'],
      dtype='object', length=1834)

In [0]:
test_data_model['retweet_count'] = test_data_model['retweet_count'].astype(float)

In [0]:
test_data_model = test_data_model.drop(columns=['id', 'original_text', 'lang', 'original_author'
, 'text', 'tokenized', 'label'], axis=1)

In [0]:
test_data_model.to_csv("test_data_model.csv")

In [0]:
RF = RandomForestClassifier(random_state=seed,n_estimators=403,max_depth=10)
RF.fit(data_model.drop(columns='label',axis=1),data_model['label'])
RF_predictions = RF.predict(test_data_model)
RF_results = pd.DataFrame([],columns=["id","sentiment_class"])
RF_results["id"] = test_model["id"].astype("int64")
RF_results["sentiment_class"] = RF_predictions
RF_results.to_csv("results_RF.csv",index=False)
RF_results["sentiment_class"].unique()
RF_results["sentiment_class"].value_counts()

0    1386
1       1
Name: sentiment_class, dtype: int64

In [0]:
BN = BernoulliNB()
BN.fit(data_model.drop(columns='label',axis=1),data_model['label'])
BN_predictions = BN.predict(test_data_model)
BN_results = pd.DataFrame([],columns=["id","sentiment_class"])
BN_results["id"] = test_model["id"].astype("int64")
BN_results["sentiment_class"] = BN_predictions
BN_results.to_csv("results_BN.csv",index=False)
BN_results["sentiment_class"].unique()
BN_results["sentiment_class"].value_counts()

 0    1062
 1     169
-1     156
Name: sentiment_class, dtype: int64

In [0]:
GB = XGBoostClassifier(seed=seed)
GB.fit(data_model.drop(columns='label',axis=1),data_model['label'])
GB_predictions = GB.predict(test_data_model)
GB_results = pd.DataFrame([],columns=["id","sentiment_class"])
GB_results["id"] = test_model["id"].astype("int64")
GB_results["sentiment_class"] = GB_predictions
GB_results.to_csv("results_GB.csv",index=False)
GB_results["sentiment_class"].unique()
GB_results["sentiment_class"].value_counts()

 0    1362
-1      17
 1       8
Name: sentiment_class, dtype: int64

In [0]:
from nltk.classify import MaxentClassifier
ME = MaxentClassifier.train(test_data_model, max_iter=30)

ValueError: ignored